In [7]:
from utils.my_llm_utils import *
from utils.my_utils import *
from utils.tavily_search_utils import web_search
#from utils.crawler_utils import crawl_url

import pandas as pd
import time
import os

In [8]:
import wandb
wandb.init(project="AI-Partnerships-agent")

wandb: Currently logged in as: mikhailry (mikhailry-paylocity) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


**Logical Flow diagram**

Flow Diagram - (https://www.mermaidchart.com/raw/68d92fe9-3162-4230-bb8f-72ef19be0604?theme=light&version=v0.1&format=svg)


In [2]:
# load the LLM

# for default llm
#llm = get_llm()

# for openai llm
#llm = get_llm(use_openai=True, model_name="gpt-4o")

# for ollama llm - [gemma3:1b, llama3.2:latest, deepseek-r1:8b, phi4:latest]
llm = get_llm(use_openai=False, model_name="gemma3:1b")


# Check which llm has been loaded
if hasattr(llm, 'model'):
	print(llm.model)  # check loaded model
else:
	print(llm)

gemma3:1b


In [8]:
csv_path = "data/AI Partnerships sample.csv"

# read the csv file
df, output_path = read_csv_with_output_path(csv_path)


In [9]:
 # Update the partnerships data
success = update_partnerships(df, output_path)


Processing partnership: Hugging Face and IBM
Searching for: Hugging Face IBM AI partnership announcement


KeyboardInterrupt: 